# Python: Expectancy

Let's try Expectancy. 

Based on:
- https://www.learningmarkets.com/determining-expectancy-in-your-trading/
- http://www.unicorn.us.com/trading/expectancy.html

In [1]:
import psycopg2
import pandas as pd
import plotly.express as px
from IPython.core.display import HTML
from IPython.display import display, Image

In [2]:
# Connect to the database
conn = psycopg2.connect(host="quantdata.collective2.com",database="postgres",user="c2public",password="c2public")
cursor = conn.cursor()

# Get max. date we have in stats 
sql = "select max(date) from historical_stats_modern;"
cursor.execute(sql)
maxDate = cursor.fetchone()

# Prepare SQL text
sqlTemplate = """select 
    r.systemid,
    c2systems.systemname, 
    ((AvgWin / AvgLoss) - (cast(numloss as float) / numtrades)) as Expectancy,
    annReturn.statval as AnnReturn,
    deltaequityp180.statval as DeltaEquityp180,
    (cast(numwins as float) / numtrades) as WinRatio,
    (cast(numloss as float) / numtrades) as LossRatio,
    AvgWin, 
    AvgLoss,
    cast(AvgWin / AvgLoss as float) as RewardToRisk,
    numtrades, 
    numwins, 
    numloss 
from c2ex_results r  
join c2systems on c2systems.systemid = r.systemid 
-- Select by AnnReturn. Based on Daniil's work.
join historical_stats_modern annReturn 
        on annReturn.systemid = r.systemid 
        and annReturn.date = '{date}'
        and annReturn.statname = 'cARdefault' 
        and annReturn.statval > 0
-- Select by DeltaEquityp180. Based on Daniil's work.        
join historical_stats_modern deltaequityp180 
        on deltaequityp180.systemid = r.systemid 
        and deltaequityp180.date = '{date}'
        and deltaequityp180.statname = 'deltaequityp180' 
        and deltaequityp180.statval > 0
where
-- Select systems live on maxDate (from equities table)
(c2systems.systemid IN ( SELECT distinct(systemid) FROM returnsdatainintervalscleanedskip090 where DateStart = '{maxDate}'))
and numtrades > 100 -- a reasonable amount for stats
and numwins > 0 -- no losers, please
and avgloss > 0 -- divider
;"""

# Format maxDate for SQL
maxDateStr = str(maxDate[0])
maxDateFormatted = maxDateStr[0:4] + "-" + maxDateStr[4:6] + "-" + maxDateStr[6:8]

# Create an SQL command from the template:
sql = sqlTemplate.format(date = maxDate[0], maxDate = maxDateStr)

# Run SQL
cursor.execute(sql)
dbData = cursor.fetchall()
cursor.close()
print("Date used:",maxDateStr)

Date used: 2021-11-29


In [3]:
# Convert dbData to pandas dataframe.
df = pd.DataFrame(dbData, 
                  columns=['SystemId','Name','Expectancy','AnnReturn','DeltaEquityp180',
                           'WinRatio','LossRatio','AvgWin','AvgLoss','RewardToRisk','NumTrades','NumWins','NumLoss'])

# Select strategies having Expectancy above some value
df = df[df['Expectancy'] > 0.2] 
# Sort by Expectancy descending
df.sort_values(by=['Expectancy'], ascending=False, inplace=True) 
# Round nubers
df = df.round(decimals=3)
display(df)

,SystemId,Name,Expectancy,AnnReturn,DeltaEquityp180,WinRatio,LossRatio,AvgWin,AvgLoss,RewardToRisk,NumTrades,NumWins,NumLoss
1,96866107,Brands With Momentum,5.382,0.38060,37.00000,0.545,0.455,1714.82,293.83,5.836,132,72,60
93,135738325,Momentum Futures,3.770,1.18200,35.80000,0.229,0.771,1691.23,372.49,4.540,266,61,205
14,117734561,TQQQ Aspire,3.043,0.56860,59.80000,0.391,0.609,2981.76,816.47,3.652,128,50,78
38,128627295,Interlink Futures,2.896,0.76920,79.70000,0.281,0.719,2556.74,707.41,3.614,430,121,309
19,124190857,ES No Guts No Glory,2.632,0.46490,0.90000,0.292,0.708,3088.02,924.45,3.340,168,49,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,125486879,My Live Portfolio,0.265,0.49440,9.40000,0.897,0.103,195.25,531.09,0.368,437,392,45
41,129020902,FOREX VIX,0.254,0.15370,46.60000,0.675,0.325,170.90,295.40,0.579,360,243,117
11,116569489,FX Alpha Signal,0.240,0.13800,8.40000,0.746,0.254,4298.37,8695.19,0.494,169,126,43
0,13202557,extreme-os,0.235,0.28010,13.20000,0.727,0.273,822.56,1619.40,0.508,4235,3077,1158


In [4]:
html = df.to_html()
file = open("expectancy.html","w")
file.write(html)
file.close()

[See expectancy.html](https://collective2.github.io/QuantData/Documentation/expectancy.html)

In [5]:
# Show the first 30 strategies here, in this notebook
for index, row in df[0:30].iterrows():
    c2url= "https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid={systemid}".format(systemid = row['SystemId'])
    img = Image(url=c2url)
    display(img)
    line = "{systemname} (#{systemid}) Expectancy: {expect}".format(systemname = row['Name'], systemid = row['SystemId'],expect=row['Expectancy']) 
    print(line)
    display(HTML("<hr/>"))


Brands With Momentum (#96866107) Expectancy: 5.382


Momentum Futures (#135738325) Expectancy: 3.77


TQQQ Aspire (#117734561) Expectancy: 3.043


Interlink Futures (#128627295) Expectancy: 2.896


ES No Guts No Glory (#124190857) Expectancy: 2.632


...

tqqqnqq (#134255352) Expectancy: 1.26


TQQQSQQQ (#124727146) Expectancy: 1.183


Star  sp500 (#135282700) Expectancy: 1.165


3Algo Futures (#128751974) Expectancy: 1.15


4Timing Trend ML (#124696549) Expectancy: 1.138
